In [1]:
from dmrApp import create_app, db
from dmrApp.models import Dmrs, Employees, Employeeroles, Post, Restaurants, Shifts, \
    User, Standardwages
import numpy as np
from datetime import date, datetime
from sqlalchemy import func
proxyApp=create_app()
ctx=proxyApp.app_context()
ctx.push()

C:\tools\miniconda3\envs\dmrAppConda\lib\site-packages\flask_sqlalchemy\__init__.py:833: FSADeprecationWarning: SQLALCHEMY_TRACK_MODIFICATIONS adds significant overhead and will be disabled by default in the future.  Set it to True or False to suppress this warning.
  warnings.warn(FSADeprecationWarning(


In [2]:
import pandas as pd
import os
from flask import current_app

In [3]:
db.create_all()

In [49]:
ctx.pop()

In [5]:
dmrDateObj=date(2021,1,9)
resId=3

In [5]:
#for each shift in shifts
shiftDataAdd=Shifts.query.filter_by(shiftDate=dmrDateObj,restaurantGroupId=2)
for i in shiftDataAdd:
    resFullTipRate=db.session.query(Dmrs.bartenderTipsPerHour,func.max(Dmrs.id)).filter_by(restaurantId=i.restaurant.id, dmrDate=dmrDateObj).first()[0]
    resHalfTipRate=db.session.query(Dmrs.appTipsPerHour,func.max(Dmrs.id)).filter_by(restaurantId=i.restaurant.id, dmrDate=dmrDateObj).first()[0]
#get the shift's restaurant's tips

0.6666666666666666
0.3333333333333333
0.6666666666666666
0.3333333333333333
1.0
0.0


In [7]:
dateFrom=date(2020,1,1)
dateTo=date(2021,1,21)
shiftData = Shifts.query.filter(Shifts.shiftDate>=dateFrom,Shifts.shiftDate<=dateTo)

In [11]:
for i in shiftData:
    print("{:.2f}".format(i.shiftTipsShipgarten) if i.restaurant.id in [3,4,5,6] else i.shiftTips)

0.67
0.33
0.67
0.33
1.0


In [12]:
for i in shiftData:
    print(i.employeerole.standardwage.wage)

2.13
2.13
2.13
2.13
2.13


In [7]:
engine=create_engine(r'sqlite:///D:\databases\dmrApp.db').connect()

In [22]:
df=pd.read_sql_table('standardwages', db.engine)

In [19]:
#get list of tables
db.engine.table_names()

['alembic_version', 'dmrs', 'employeeroles', 'employees', 'post', 'restaurants', 'shifts', 'standardwages', 'user']


In [21]:
print(db.engine)

Engine(sqlite:///D:\databases\dmrApp.db)


In [40]:
formDict={'dmrs': 'True', 'employeeroles': 'True', 'employees': 'True', 'downloadTables': 'True'}

In [42]:
list(formDict.keys())[:-1]

['dmrs', 'employeeroles', 'employees']

In [26]:
dfDictionary={h : pd.read_sql_table(h, db.engine) for h in formDict}

In [43]:
for h, i in dfDictionary.items():
    print(len(i))

26
6
4


In [30]:
dfDictionary['employeeroles']

,id,role,wage,tipPercentage,notes,standardWageFlag,createdBy,restaurantId,standardWageId,empId
0,1,bartender,2.13,1.0,None,True,1,3,1,1
1,2,security,12.00,0.0,None,True,1,3,3,1
2,3,bartender,2.13,1.0,None,True,1,3,1,2
3,4,kitchen,12.00,0.0,None,True,1,3,5,2
4,5,apprentice,2.13,0.5,None,True,1,3,2,3
5,6,apprentice,2.13,0.5,None,True,1,4,2,4


In [14]:
path=r'C:\Users\captian2020\Downloads'

In [20]:
postUpload=pd.read_excel(os.path.join(path,"databaseTablesDownload.xlsx"),engine='openpyxl',sheet_name='post')

In [39]:
postUpload2=pd.read_excel(os.path.join(path,"databaseTablesDownload.xlsx"),engine='openpyxl')

In [23]:
postUpload.to_sql('post',con=db.engine, if_exists='append', index=False)

In [25]:
postUpload

,id,title,date_posted,content,user_id
0,5,version test upload,2021-01-09 16:31:55.274568,test,1


In [40]:
postUpload2.keys()

Index(['id', 'title', 'date_posted', 'content', 'user_id'], dtype='object')

In [35]:
dfPost=pd.read_excel(os.path.join(path,"databaseTablesDownload.xlsx"),none)

NameError: name 'none' is not defined

In [42]:
import openpyxl
wb = openpyxl.load_workbook(os.path.join(path,"databaseTablesDownload.xlsx")) 

In [61]:
sheetNames=wb.sheetnames
sheetNames

['post', 'Sheet1']

In [46]:
uploadDict={i: pd.read_excel(os.path.join(
    path,"databaseTablesDownload.xlsx"),engine='openpyxl',sheet_name=i) for i in sheetNames}

In [47]:
uploadDict['post']

,id,title,date_posted,content,user_id
0,5,version test upload,2021-01-09 16:31:55.274568,test,1


In [48]:
tableDict={'employees':''}

In [55]:
dfDictionary={h : pd.read_sql_table(h, db.engine) for h in tableDict}

In [56]:
print(dfDictionary.get('employees'))

   id         name                  timeStamp  createdBy
0   1         Nick 2021-01-01 11:57:35.891261          1
1   2       Shayan 2021-01-02 19:19:10.107122          1
2   3  apprentice1 2021-01-08 14:44:25.560039          1
3   4  apprentice2 2021-01-08 14:47:01.874884          1


In [59]:
dfDictionary2={h : dfDictionary.get(h).reset_index(drop=True, inplace=True) for h in dfDictionary}

In [60]:
dfDictionary2.get('employees')

In [63]:
tableNamesList= db.engine.table_names()
tableNamesList[3]

'employees'

In [72]:
db.session.query(dbModelsList[3]).delete()
db.session.commit()

In [74]:
# dbTest = current_app.extensions['sqlalchemy'].db

dbModelsList= [cls for cls in db.Model._decl_class_registry.values() if isinstance(cls, type) and issubclass(cls, db.Model)]

In [76]:
dbModelsList

[dmrApp.models.Dmrs,
 dmrApp.models.Shifts,
 dmrApp.models.Restaurants,
 dmrApp.models.Employees,
 dmrApp.models.Employeeroles,
 dmrApp.models.Standardwages,
 dmrApp.models.User,
 dmrApp.models.Post]

In [83]:
dbModelsDict={str(h)[22:-2]:h for h in dbModelsList}

In [84]:
dbModelsDict

{'Dmrs': dmrApp.models.Dmrs,
 'Shifts': dmrApp.models.Shifts,
 'Restaurants': dmrApp.models.Restaurants,
 'Employees': dmrApp.models.Employees,
 'Employeeroles': dmrApp.models.Employeeroles,
 'Standardwages': dmrApp.models.Standardwages,
 'User': dmrApp.models.User,
 'Post': dmrApp.models.Post}

In [30]:
dmrDateObj=date(2021,1,9)

In [28]:
    dmrIdSg = Dmrs.query.filter(Dmrs.restaurantId==2,Dmrs.dmrDate>date(2020,1,1)).order_by(Dmrs.id.desc()).first()
    dmrIdSg

Dmr('3','2021-01-09','2','Shipgarten')

In [16]:
type(dmrIdSg.dmrDate)

datetime.date

In [17]:
type(dmrDateObj)

datetime.date

In [21]:
dateCheck=dmrIdSg.dmrDate

In [33]:
Dmrs.query.filter(Dmrs.restaurantId==2,Dmrs.dmrDatedmrDateObj).with_entities(
Dmrs.id).order_by(Dmrs.id.desc()).first()[0]

3

In [35]:
db.session.commit()

In [36]:
colNamesShifts =['Date','Role','Restaurant','Name','Scheduled time','Start time',
    'Off time','Hours worked','Tips', 'Wages', 'Tips + Wages']
dateFrom=date(2021,1,1)
dateTo=date(2021,2,1)

In [37]:
def shiftDataReport(dateFrom,dateTo, colNamesShifts):
    shiftData = Shifts.query.filter(Shifts.shiftDate>=dateFrom,
                                    Shifts.shiftDate<=dateTo)
    shiftReportColumns = list(shiftDataLists(shiftData))
    dfShifts = pd.DataFrame(list(zip(shiftReportColumns[0],shiftReportColumns[1],
                               shiftReportColumns[2],shiftReportColumns[3],
                               shiftReportColumns[4],shiftReportColumns[5],
                               shiftReportColumns[6],shiftReportColumns[7],
                               shiftReportColumns[8]
                               )), columns=colNamesShifts)
    return dfShifts

In [38]:
shiftDataReport(dateFrom,dateTo, colNamesShifts)

NameError: name 'shiftDataLists' is not defined

In [40]:
datetime.now()

datetime.datetime(2021, 1, 12, 12, 53, 1, 23185)

In [41]:
shiftData = Shifts.query.filter(Shifts.shiftDate>=dateFrom,Shifts.shiftDate<=dateTo)

In [42]:
shiftData.all()

[Shift('1','','empId: 1','Nick','Tysons Biergarten'),
 Shift('2','','empId: 2','Shayan','Tysons Biergarten'),
 Shift('3','','empId: 3','apprentice1','Tysons Biergarten'),
 Shift('4','','empId: 3','apprentice1','Salamati Grill')]

In [51]:
shiftEmpIdList=shiftData.with_entities(Shifts.empId).distinct()
shiftEmpIdList.all()

[(1), (2), (3)]

In [45]:

db.session.query(Shifts).group_by(Shifts.empId).with_entities(func.sum(Shifts.shiftTips)).all()

[(1.0), (0.3333333333333333), (1.6666666666666665)]